In [2]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from scipy.odr import ODR, Model, RealData

from dotenv import load_dotenv
load_dotenv(override=True)

ROOT_PATH = os.environ.get('ROOT_PATH')
if not ROOT_PATH in sys.path: sys.path.append(ROOT_PATH)

from main_code.utils.constants import *
from main_code.utils.CosmoFunc import *
from main_code.utils.functions import density_contour

# def bin_data(x: np.array, y: np.array, xmin: float, xmax: float, n_bin: int):
#     # x_bin = np.linspace(np.min(x), np.max(x), n_bin)
#     x_bin = np.linspace(xmin, xmax, n_bin)
#     x_middle = 0.5 * (x_bin[1:] + x_bin[:-1])
#     delta_x = np.diff(x_bin)[0]

#     x_bin_ = []
#     y_bin = []
#     y_bin_err = []
#     y_bin_stderr = []

#     for x_trial in x_middle:
#         x_lower = x_trial - 0.5 * delta_x 
#         x_upper = x_trial + 0.5 * delta_x

#         y_ = y[(x >= x_lower) & (x < x_upper)]

#         if len(y_):
#             x_bin_.append(x_trial)
#             y_bin.append(np.median(y_))
#             y_bin_err.append(np.std(y_))
#             y_bin_stderr.append(np.std(y_) / np.sqrt(len(y_)))
#         else:
#             continue

#     return np.array(x_bin_), np.array(y_bin), np.array(y_bin_err), np.array(y_bin_stderr)

In [2]:
import sys
sys.path

['/Users/mrafifrbbn/anaconda3/lib/python311.zip',
 '/Users/mrafifrbbn/anaconda3/lib/python3.11',
 '/Users/mrafifrbbn/anaconda3/lib/python3.11/lib-dynload',
 '',
 '/Users/mrafifrbbn/Documents/thesis/thesis-research-2.0/venv/lib/python3.11/site-packages']

In [5]:
!which python

/Users/mrafifrbbn/Documents/thesis/thesis-research-2.0/venv/bin/python
